In [9]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import keras
import tensorflow as tf
import pandas as pd
import datetime
import keras.backend as K
K.clear_session()
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


First let's import keras and some other libraries

In [19]:
 data = pd.read_csv("m_stroman_pitches_v3.csv")

let's load the pitches. above are 3045 pitches, that marcus stroman threw in sequence from 2019

In [20]:
df = pd.DataFrame(data)

pitches = df['pitch_type']
pitches = pd.get_dummies(pitches) #get a one hot encoding for pitches
p_enc = pitches.to_numpy()

del df['pitch_type'] #don't need pitch type anymore

In [21]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

#label encode stance right or left
df["stand"] = le.fit_transform(df["stand"])

#one hot encode previous type, ball/strike/or in play
ty = pd.get_dummies(df['prev_type'])
df = df.drop('prev_type',axis = 1)
df = df.join(ty)

#one hot encode previous pitch string
prevs = pd.get_dummies(df['prev_pitch'])
df = df.drop('prev_pitch',axis = 1)
# Join the encoded df
df = df.join(prevs)
variables = df.to_numpy()

In [22]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split( 
    variables, p_enc, test_size=0.2)

In [23]:
model = Sequential()
from keras.optimizers import SGD
model.add(Dense(8, activation = 'relu', input_dim = 13))
model.add(Dense(3, activation = 'softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(trainX, trainY, epochs = 100,
            batch_size = 64)

Epoch 1/100
2435/2435 [==============================] - 0s 66us/step - loss: 0.9911 - accuracy: 0.5433
Epoch 2/100
2435/2435 [==============================] - 0s 12us/step - loss: 0.8627 - accuracy: 0.5741
Epoch 3/100
2435/2435 [==============================] - 0s 17us/step - loss: 0.8481 - accuracy: 0.5634
Epoch 4/100
2435/2435 [==============================] - 0s 25us/step - loss: 0.8464 - accuracy: 0.5795
Epoch 5/100
2435/2435 [==============================] - 0s 30us/step - loss: 0.8420 - accuracy: 0.5819
Epoch 6/100
2435/2435 [==============================] - 0s 31us/step - loss: 0.8329 - accuracy: 0.5807
Epoch 7/100
2435/2435 [==============================] - 0s 32us/step - loss: 0.8329 - accuracy: 0.5803
Epoch 8/100
2435/2435 [==============================] - 0s 30us/step - loss: 0.8352 - accuracy: 0.5840
Epoch 9/100
2435/2435 [==============================] - 0s 31us/step - loss: 0.8309 - accuracy: 0.5832
Epoch 10/100
2435/2435 [==============================] - 0s 30u

2435/2435 [==============================] - 0s 31us/step - loss: 0.7946 - accuracy: 0.6000
Epoch 79/100
2435/2435 [==============================] - 0s 29us/step - loss: 0.7968 - accuracy: 0.6033
Epoch 80/100
2435/2435 [==============================] - 0s 29us/step - loss: 0.7955 - accuracy: 0.5996
Epoch 81/100
2435/2435 [==============================] - 0s 29us/step - loss: 0.7998 - accuracy: 0.6033
Epoch 82/100
2435/2435 [==============================] - 0s 29us/step - loss: 0.7984 - accuracy: 0.6025
Epoch 83/100
2435/2435 [==============================] - 0s 29us/step - loss: 0.7981 - accuracy: 0.6053
Epoch 84/100
2435/2435 [==============================] - 0s 30us/step - loss: 0.7977 - accuracy: 0.5979
Epoch 85/100
2435/2435 [==============================] - 0s 29us/step - loss: 0.7997 - accuracy: 0.5943
Epoch 86/100
2435/2435 [==============================] - 0s 29us/step - loss: 0.7958 - accuracy: 0.5996
Epoch 87/100
2435/2435 [==============================] - 0s 30us/st

In [24]:
score = model.evaluate(testX, testY, batch_size = 32)
print("Test Accuracy: "+"{:.2%}".format(score[1]))

609/609 [==============================] - 0s 121us/step
Test Accuracy: 57.31%


In [18]:
f = open("file.txt", "a")
import sys
np.set_printoptions(threshold=sys.maxsize)
f.write(np.array2string(model.predict(testX)))
f.close()